# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
import lightgbm as lgb

# Load Data

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

# Data Preprocessing

In [3]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


In [4]:
## x 의 경우도 범주형으로 구성되어 있어, 알맞은 인코딩 필요
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

categorical_columns = X.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = X.copy()
X_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

In [18]:
X_encoded.shape

(2546, 4384)

# Model Define and Train

In [9]:
model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.01,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss' 
)


In [10]:
model.fit(X_encoded, y_subclass)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023607 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 75427
[LightGBM] [Info] Number of data points in the train set: 6201, number of used features: 3205
[LightGBM] [Info] Start training from score -4.455800
[LightGBM] [Info] Start training from score -4.088075
[LightGBM] [Info] Start training from score -2.065509
[LightGBM] [Info] Start training from score -3.689041
[LightGBM] [Info] Start training from score -3.325294
[LightGBM] [Info] Start training from score -5.094880
[LightGBM] [Info] Start training from score -2.599068
[LightGBM] [Info] Start training from score -3.

LGBMClassifier(eval_metric='mlogloss', learning_rate=0.01, max_depth=6,
               random_state=42, use_label_encoder=False)

# Inference

In [11]:
test_X = test.drop(columns=['ID'])
X_encoded = test_X.copy()
X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])

In [12]:
predictions = model.predict(X_encoded)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric


In [13]:
original_labels = le_subclass.inverse_transform(predictions)

# Submisson

In [14]:
submisson = pd.read_csv("./sample_submission.csv")

In [16]:
submisson["SUBCLASS"] = original_labels
submisson["SUBCLASS"].value_counts()

SUBCLASS
BRCA      851
COAD      380
GBMLGG    210
STES      172
KIPAN     153
OV        134
THCA       77
UCEC       68
HNSC       62
SKCM       55
DLBC       48
LUAD       35
PAAD       33
LGG        33
LIHC       33
LAML       26
KIRC       26
TGCT       24
CESC       22
ACC        18
BLCA       18
LUSC       17
PCPG       16
PRAD       13
THYM       12
SARC       10
Name: count, dtype: int64

In [17]:
submisson.to_csv('./baseline_submission_lgb.csv', encoding='UTF-8-sig', index=False)